In [1]:
!pip install yfinance pandas numpy tensorflow scikit-learn


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data

In [6]:
def preprocess_data(stock_data, column='Close'):
    close_prices = stock_data[column].values
    close_prices = close_prices.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)
    def create_dataset(data, time_step=60):
        X, y = [], []
        for i in range(time_step, len(data)):
            X.append(data[i - time_step:i, 0])
            y.append(data[i, 0])
        return np.array(X), np.array(y)
    X, y = create_dataset(scaled_data)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y, scaler


In [7]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [8]:
def train_lstm_model(ticker, start_date, end_date):
    stock_data = fetch_stock_data(ticker, start_date, end_date)
    X, y, scaler = preprocess_data(stock_data)
    model = create_lstm_model(X.shape[1:])
    model.fit(X, y, epochs=10, batch_size=32)
    return model, scaler, stock_data

In [9]:
def predict_future_stock(model, scaler, stock_data, days=30):
    last_60_days = stock_data['Close'][-60:].values
    last_60_days = last_60_days.reshape(-1, 1)
    scaled_last_60_days = scaler.transform(last_60_days)
    X_input = []
    X_input.append(scaled_last_60_days)
    X_input = np.array(X_input)
    X_input = X_input.reshape(X_input.shape[0], X_input.shape[1], 1)
    predicted_price = model.predict(X_input)
    predicted_price = scaler.inverse_transform(predicted_price)

    return predicted_price

In [10]:
def plot_predictions(stock_data, predicted_price):
    plt.figure(figsize=(14,7))
    plt.plot(stock_data['Close'], label='Actual Stock Price')
    plt.plot(len(stock_data) + 1, predicted_price, marker='o', color='red', label='Predicted Price')
    plt.title('Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

In [15]:
ticker = 'GOOGL'
start_date = '2020-01-01'
end_date = '2023-01-01'

In [16]:
model, scaler, stock_data = train_lstm_model(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed

Epoch 1/10



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - loss: 0.1939
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0115
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0043
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0023
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0023
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0020
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0020
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - loss: 0.0021
Epoch 9/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0022
Epoch 10/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0020


In [17]:
predicted_price = predict_future_stock(model, scaler, stock_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
